# P2

## get the data:

In [1]:
# Load necessary library
library(quantmod)

# Set seed for reproducibility
set.seed(53600945)

# Generate 100 random integers within the range 1 to 377
random_indices <- sample(1:377, 100, replace = FALSE)

# Read the tickers from the "symbols2.csv" file
symbols_df <- read.csv("symbols2.csv")

# Select tickers based on the generated random integers
selected_tickers <- symbols_df$Ticker[random_indices]

# Define the start and end dates for the data download
start_date <- as.Date("2020-01-01")
end_date <- as.Date("2022-10-31")

# Initialize a list to store the stock data
stock_data_list <- list()

# Loop through the selected tickers and download the stock price data
for(ticker in selected_tickers) {
  # The tryCatch function is used to ignore errors (e.g., if the stock is not available)
  stock_data <- tryCatch({
    getSymbols(ticker, src = 'yahoo', from = start_date, to = end_date, auto.assign = FALSE)
  }, error = function(e) NULL)
  
  # If stock data was successfully downloaded, add it to the list
  if (!is.null(stock_data)) {
    stock_data_list[[ticker]] <- stock_data
  }
}

# At this point, `stock_data_list` contains the downloaded stock data for each available ticker
# You can access each stock's data using stock_data_list[['Ticker']]


Loading required package: xts
Loading required package: zoo

Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.


## compute log returns:

In [2]:
# Function to compute logarithmic returns
compute_log_returns <- function(stock_prices) {
  # Compute logarithmic returns using diff and log
  log_returns <- diff(log(Cl(stock_prices)))
  return(log_returns)
}

# Initialize a list to store the logarithmic returns for each stock
log_returns_list <- list()

# Loop through each stock's data in `stock_data_list` to compute the logarithmic returns
for(ticker in names(stock_data_list)) {
  stock_prices <- stock_data_list[[ticker]]
  log_returns <- compute_log_returns(stock_prices)
  log_returns_list[[ticker]] <- log_returns
}

# At this point, `log_returns_list` contains the logarithmic returns for each available ticker
# You can access the logarithmic returns of each stock using log_returns_list[['Ticker']]


## compute mean log returns:

In [6]:
# Assuming `log_returns_list` contains the logarithmic returns for each stock

# Combine all logarithmic returns into a single data frame
# Each column in this data frame will represent a stock, and each row represents a date
combined_log_returns <- do.call(merge, c(log_returns_list, all = TRUE))

# Compute the mean logarithmic return for each date
mean_log_returns <- rowMeans(combined_log_returns, na.rm = TRUE)

# Convert the mean logarithmic returns to a data frame for easier handling
mean_log_returns_df <- data.frame(Date = index(combined_log_returns), MeanLogReturn = mean_log_returns)

# Preview the data frame containing the mean logarithmic returns
head(mean_log_returns_df)


ERROR: Error in as.data.frame(x): argument "x" is missing, with no default
